This notebook demonstrates a workflow for converting a portfolio amount in USD to a target currency (INR or EUR) using a state graph. The workflow is structured as follows:

1. **Define Portfolio State**: A `TypedDict` is used to define the structure of the portfolio state, including fields for the amount in USD, total in USD, target currency, and the final converted total.

2. **Conversion Functions**: Functions are implemented to calculate the total in USD and convert it to the target currency (INR or EUR). A helper function determines the target currency for conversion.

3. **State Graph Construction**: Using the `StateGraph` class, a graph is built to represent the workflow. Nodes are added for each step in the process, and conditional edges are used to route the workflow based on the target currency.

4. **Graph Visualization**: The constructed graph is visualized using Mermaid diagrams to provide a clear representation of the workflow.

5. **User Interaction**: The user is prompted to input the amount in USD and the target currency. The graph is invoked with the provided inputs, and the workflow executes the appropriate conversion.

This notebook combines Python programming, state graph modeling, and interactive user input to demonstrate a structured approach to currency conversion.

In [ ]:
from typing import TypedDict, Literal

class PortfolioState(TypedDict):
    amount_usd: float
    total_usd: float
    target_currency: Literal['inr', 'eur']
    total: float

In [ ]:
def calc_total(state: PortfolioState) -> PortfolioState:
    state['total_usd'] = state['amount_usd'] * 1.08
    return state

def conv_inr(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd'] * 85.69
    return state

def conv_eur(state: PortfolioState) -> PortfolioState:
    state['total'] = state['total_usd'] * 0.85
    return state

def choose_conversion(state: PortfolioState) -> str:
    return state['target_currency']

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(PortfolioState)

builder.add_node("calc_total_node", calc_total)
builder.add_node("conv_inr_node", conv_inr)
builder.add_node("conv_eur_node", conv_eur)

builder.add_edge(START, "calc_total_node")
builder.add_conditional_edges(
    "calc_total_node",
    choose_conversion, {
    "inr": "conv_inr_node",
    "eur": "conv_eur_node"
})
builder.add_edge(["conv_inr_node", "conv_eur_node"], END)


graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
amt = input("Enter amount in USD: ")

graph.invoke(
    {
        "amount_usd": float(amt),
        "target_currency": input("Enter target currency (inr/eur): ")
    }
)